# European Soccer Database

Exploratory Data Analysis on the European Soccer Database

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore")
color = sns.color_palette()
%matplotlib inline

In [4]:
# connect the soccer database
conn = sqlite3.connect('database.sqlite')

 ### Reading the tables
 We'll start by reading the SQL tables into Pandas dataframes. <br/>
 Starting with all tables:

In [6]:
# read the tables
tables = pd.read_sql("""
    SELECT *
    FROM sqlite_master
    WHERE type='table';
""", conn)

# print the tables
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


Now that we can see the tables that are available to us, we can start checking them out. We can first look at European countries that are represented in this database.

In [8]:
# read the countries table
countries = pd.read_sql("""
    SELECT *
    FROM Country;
""", conn)

# print the countries table
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


As we can see, there is good selection on countries, including the top ones and some lower tier countries. Now we should check out the leagues in these countries.

In [9]:
leagues = pd.read_sql("""
    SELECT *
    FROM League;
""", conn)

leagues

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


Only the country id is present in this table so we should also join the countries table here based on the ID for a better view of the table.

In [34]:
leagues = pd.read_sql("""
    SELECT *
    FROM League
    JOIN Country ON Country.id = League.country_id;
""", conn)

# rename the columns
leagues.columns = ['id', 'country_id', 'league_name', 'id_1', 'country_name']
# drop the second id column
leagues = leagues.drop(['id_1'], axis=1)
leagues

,id,country_id,league_name,country_name
0,1,1,Belgium Jupiler League,Belgium
1,1729,1729,England Premier League,England
2,4769,4769,France Ligue 1,France
3,7809,7809,Germany 1. Bundesliga,Germany
4,10257,10257,Italy Serie A,Italy
5,13274,13274,Netherlands Eredivisie,Netherlands
6,15722,15722,Poland Ekstraklasa,Poland
7,17642,17642,Portugal Liga ZON Sagres,Portugal
8,19694,19694,Scotland Premier League,Scotland
9,21518,21518,Spain LIGA BBVA,Spain


This is better. We can see that just the first divisions in each country are in the database. Probably, there isn't much data on the lower tiers, but it could have been interesting to analyse those as well, especially the lower tier English division (Championship). <br/>
Let's move to the teams table.

In [37]:
teams = pd.read_sql("""
    SELECT *
    FROM Team
    ORDER BY team_long_name;
""", conn)

teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,16848,8350,29.0,1. FC Kaiserslautern,KAI
1,15624,8722,31.0,1. FC Köln,FCK
2,16239,8165,171.0,1. FC Nürnberg,NUR
3,16243,9905,169.0,1. FSV Mainz 05,MAI
4,11817,8576,614.0,AC Ajaccio,AJA


Now on to the actual matches, the interesting table from this database.

In [39]:
matches = pd.read_sql("""
    SELECT *
    FROM Match;
""", conn)

matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
